In [1]:
import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer

Using TensorFlow backend.


[]


In [2]:
vec = cPickle.load(open('../../yelpdata/total_vec_120K_embed.p', 'rb'))

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

ds = pd.read_csv('../../yelpdata/total_data_120K.csv')
aspect_columns = ['bit', 'domain']
                 
ds['bit'] = ds['stars'].apply(lambda x : x > 3.0)
train_idxs, val_idxs = train_test_split(ds.index, stratify=ds[['bit', 'domain']], train_size=0.9, random_state=1337)

idxs = val_idxs
H = {}
for aspect in aspect_columns :
    H[aspect] = np.zeros((len(idxs), len(idxs)))
    aspect_vals = ds[aspect].unique()
    for val in aspect_vals :
        a = set(ds[ds[aspect] == val].index) & set(idxs)
        a = map(lambda s : list(idxs).index(s), a)
        for j in a :
            H[aspect][j, a] = 1

    H[aspect][np.arange(len(idxs)), np.arange(len(idxs))] = 0

/home/sarthak/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [4]:
train_X = vec.X[train_idxs]
train_X = map(lambda s : [vec.idx2word[t] for t in s if t > 0], train_X)

In [5]:
val_X = vec.X[val_idxs]
val_X = map(lambda s : [vec.idx2word[t] for t in s if t > 0], val_X)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=lambda i:i, lowercase=False)
train_TF = tfidf.fit_transform(train_X)

In [23]:
val_TF = tfidf.transform(val_X)

In [26]:
#TF
from sklearn.preprocessing import normalize
nb_studies = len(val_idxs)
embedds_n = normalize(val_C, 'l2')
scores = np.dot(embedds_n, embedds_n.T)
nb_studies = len(val_idxs)
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000

In [25]:
#C
from sklearn.metrics import roc_auc_score
scores = np.array(scores.todense())
for aspect_j in H :
    aucs = [0] * nb_studies
    for i in range(nb_studies) :
        aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
    print aspect_j, np.mean(aucs)

bit 0.523671227626
domain 0.595010542726


In [27]:
from sklearn.metrics import roc_auc_score
scores = np.array(scores.todense())
for aspect_j in H :
    aucs = [0] * nb_studies
    for i in range(nb_studies) :
        aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
    print aspect_j, np.mean(aucs)

bit 0.514199859529
domain 0.568231361097


In [17]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=3)
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(tokenizer=lambda i:i, lowercase=False)
train_C = count.fit_transform(train_X)
val_C = count.transform(val_X)

In [18]:
train_lda = lda.fit_transform(train_C)

/home/sarthak/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [19]:
val_lda = lda.transform(val_C)

In [20]:
from sklearn.preprocessing import normalize
nb_studies = len(val_idxs)
embedds_n = normalize(val_lda, 'l2')
scores = np.dot(embedds_n, embedds_n.T)
nb_studies = len(val_idxs)
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000

In [11]:
from sklearn.metrics import roc_auc_score
scores = np.array(scores)
for aspect_j in H :
    aucs = [0] * nb_studies
    for i in range(nb_studies) :
        aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
    print aspect_j, np.mean(aucs)

bit 0.617423372048
domain 0.896471856586


In [16]:
from sklearn.metrics import roc_auc_score
scores = np.array(scores)
for aspect_j in H :
    aucs = [0] * nb_studies
    for i in range(nb_studies) :
        aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
    print aspect_j, np.mean(aucs)

bit 0.614150028443
domain 0.875290433066


In [21]:
from sklearn.metrics import roc_auc_score
scores = np.array(scores)
for aspect_j in H :
    aucs = [0] * nb_studies
    for i in range(nb_studies) :
        aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
    print aspect_j, np.mean(aucs)

bit 0.596580963023
domain 0.936841578582


In [ ]:
train_lsa = lsa.fit_transform(train_TF)

In [ ]:
val_lsa = lsa.transform(val_TF)